In [ ]:
from scipy import stats
import pandas as pd
import plotly.express as px
import numpy as np
from scipy.stats import gamma
import statsmodels.api as sm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from numpy.random import RandomState
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use("ggplot")

In [ ]:
# set random seed and random number generator
prng = RandomState(9)

In [ ]:
sd = 1.5
mean = 4.0

var = sd ** 2.0
scale = var / mean
a = mean / scale

In [ ]:
prng.gamma(mean, scale=sd, size=8)

In [ ]:
manual_data_for_dist = [1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 8, 9, 9, 9, 9]

In [ ]:
kde = sm.nonparametric.KDEUnivariate(manual_data_for_dist)
kde.fit()

In [ ]:
axes[0].text?

In [ ]:
x_vals = np.linspace(0.0, 15.0, 50)
epi_vals = [0, 0, 1, 3, 4]
epi_times = range(len(epi_vals))
colours = ["k", "b", "g", "r", "m"]
val_scaler = 15.0

fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
fig.tight_layout()
for i_ax, ax in enumerate(axes):
    ax.set_xlim([0.0, 15.0])
    ax.plot(epi_times, epi_vals, color="k", linestyle="--")
    for i in epi_times[2:]:
        ax.plot(i, epi_vals[i], marker="o", color=colours[i])
        ax.arrow(i + 0.5, epi_vals[i], 1.0, 0, color=colours[i], head_width=0.2)
axes[0].set_xlabel("time (days)")
axes[1].set_xlabel("time (days)")
axes[0].set_ylabel("cases (for time series)")
axes[0].set_title("assumed")
axes[1].set_title("empiric") 
distri = pd.Series(gamma.cdf(x_vals, a=a, scale=scale), index=x_vals).diff()
kernel = pd.Series(kde.density, index=kde.support)
x_val_est = 7.0
for i in epi_times[2:][::-1]:
    scaler = epi_vals[i] * val_scaler
    point = np.interp([x_val_est - i], distri.index, distri)[0] * scaler
    axes[0].fill_between(distri.index + i, distri * scaler, color=colours[i], alpha=0.1)
    axes[0].plot(distri.index + i, distri * scaler, color=colours[i], alpha=0.2)
    axes[0].plot([x_val_est], [point], marker="o", color=colours[i])
    axes[0].plot([x_val_est, x_val_est], [0.0, point], color=colours[i], linestyle=":")
    axes[0].annotate("cases\nepicurve", [0.4, 2], size="large", color=[0.3, 0.3, 0.3])
    axes[0].text(8.5, 1, "secondary\ndistributions", fontdict={"fontsize": "large", "color":[0.3, 0.3, 0.3]})
    axes[0].text(6.5, 1.5, "contributions", fontdict={"fontsize": "large", "color":[0.3, 0.3, 0.3], "rotation": 90})

    scaler /= 2.2
    point = np.interp([x_val_est - i], kernel.index, kernel)[0] * scaler
    axes[1].fill_between(kernel.index + i, kernel * scaler, color=colours[i], alpha=0.1)
    axes[1].plot(kernel.index + i, kernel * scaler, color=colours[i], alpha=0.2)
    axes[1].plot([x_val_est], [point], marker="o", color=colours[i])
    axes[1].plot([x_val_est, x_val_est], [0.0, point], color=colours[i], linestyle=":")

In [ ]:
fig.savefig("ideas_fig4.jpg")